In [ ]:
def backtest_indicators(archive_data):
    """
    Backtest technical indicators using archived data
    Returns performance metrics for each signal type
    """
    results = {
        'trend': {'correct': 0, 'total': 0},
        'momentum': {'correct': 0, 'total': 0},
        'volatility': {'correct': 0, 'total': 0},
        'volume': {'correct': 0, 'total': 0}
    }
    
    # Convert archive_data to DataFrame if it's not already
    if not isinstance(archive_data, pd.DataFrame):
        archive_df = pd.DataFrame(archive_data)
    else:
        archive_df = archive_data
        
    # Sort by date to ensure chronological order
    archive_df = archive_df.sort_values('timestamp')
    
    # Calculate forward returns (you can adjust the forward period)
    archive_df['forward_return'] = archive_df['close'].pct_change(periods=1).shift(-1)
    
    # Evaluate each signal type
    for idx in range(len(archive_df)-1):  # -1 to avoid the last row where we don't have forward returns
        row = archive_df.iloc[idx]
        
        # Trend signals evaluation
        if not pd.isna(row['trend_score']):
            results['trend']['total'] += 1
            if (row['trend_score'] > 0 and row['forward_return'] > 0) or \
               (row['trend_score'] < 0 and row['forward_return'] < 0):
                results['trend']['correct'] += 1
        
        # Momentum signals evaluation
        if not pd.isna(row['momentum_score']):
            results['momentum']['total'] += 1
            if (row['momentum_score'] > 0 and row['forward_return'] > 0) or \
               (row['momentum_score'] < 0 and row['forward_return'] < 0):
                results['momentum']['correct'] += 1
        
        # Volatility signals evaluation
        if not pd.isna(row['volatility_score']):
            results['volatility']['total'] += 1
            if (row['volatility_score'] > 0 and row['forward_return'] > 0) or \
               (row['volatility_score'] < 0 and row['forward_return'] < 0):
                results['volatility']['correct'] += 1
        
        # Volume signals evaluation
        if not pd.isna(row['volume_score']):
            results['volume']['total'] += 1
            if (row['volume_score'] > 0 and row['forward_return'] > 0) or \
               (row['volume_score'] < 0 and row['forward_return'] < 0):
                results['volume']['correct'] += 1

    # Calculate accuracy for each signal type
    performance_metrics = {}
    for signal_type, counts in results.items():
        if counts['total'] > 0:
            accuracy = (counts['correct'] / counts['total']) * 100
            performance_metrics[signal_type] = {
                'accuracy': accuracy,
                'correct_predictions': counts['correct'],
                'total_signals': counts['total']
            }
        else:
            performance_metrics[signal_type] = {
                'accuracy': 0,
                'correct_predictions': 0,
                'total_signals': 0
            }

    return performance_metrics

def print_backtest_results(performance_metrics):
    """
    Print formatted backtest results
    """
    print("\nBacktesting Results:")
    print("-" * 50)
    for signal_type, metrics in performance_metrics.items():
        print(f"\n{signal_type.capitalize()} Signals:")
        print(f"Accuracy: {metrics['accuracy']:.2f}%")
        print(f"Correct Predictions: {metrics['correct_predictions']}")
        print(f"Total Signals: {metrics['total_signals']}")

In [ ]:
# Assuming your archive data is stored in a variable called 'archive'
results = backtest_indicators(archive)
print_backtest_results(results)

# Optional: For more detailed analysis
def analyze_signal_combinations(archive_data):
    """
    Analyze the performance of combined signals
    """
    if not isinstance(archive_data, pd.DataFrame):
        df = pd.DataFrame(archive_data)
    else:
        df = archive_data
    
    df['combined_score'] = (df['trend_score'] + 
                           df['momentum_score'] + 
                           df['volatility_score'] + 
                           df['volume_score']) / 4
    
    df['forward_return'] = df['close'].pct_change(periods=1).shift(-1)
    
    # Analyze strong signals (absolute value > 0.5)
    strong_signals = df[abs(df['combined_score']) > 0.5]
    
    correct_strong = sum((strong_signals['combined_score'] > 0) & 
                        (strong_signals['forward_return'] > 0) |
                        (strong_signals['combined_score'] < 0) & 
                        (strong_signals['forward_return'] < 0))
    
    if len(strong_signals) > 0:
        strong_accuracy = (correct_strong / len(strong_signals)) * 100
    else:
        strong_accuracy = 0
    
    return {
        'strong_signal_accuracy': strong_accuracy,
        'strong_signal_count': len(strong_signals),
        'total_signals': len(df)
    }